In [ ]:
import ollama

In [ ]:
ollama.list()

In [ ]:
response = ollama.chat(
    model='smollm2:135m',
    messages=[
        {
            'role': 'user',
            'content': 'Scooptity whoop scoopity whoop whoopity scoop whoopity scoopity whoop whoop  scoopity whoop'
        }
    ]
)

print(response.message.content)

In [ ]:
response = ollama.chat(model='smollm2:135m',
     messages=[
         {'role': 'user', 
          'content': 'What is Generative AI?'}
     ])

In [ ]:
Markdown(response.message.content)

from ollama import chat

def generate_email(subject, recipient_name, additional_info):
    
    prompt = f"Write a professional email to {recipient_name} with the subject '{subject}'. Include the following information: {additional_info}"
    
    response = chat(
        model="smollm2:135m",  
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ])
    return response.message.content

In [ ]:
model = 'smollm2:135m'
prompt = 'What is a large language model?'

stream= chat(
    model=model,
    messages=[
        {'role': 'user', 'content': prompt}
    ],
    stream=True
)

for chunk in stream:
    print(chunk['message']['content'], end='')



In [ ]:
prompt_1 = "what is a large language model?"
response = chat(model='smollm2:135m',
     messages=[
         {'role': 'user', 
          'content': prompt_1}
     ])

history = response.message.content

new_prompt = 'What was the question that I asked?'

chains = prompt_1 + history + new_prompt 

response = chat(model='smollm2:135m',
     messages=[
         {'role': 'user',
          'content': chains}
     ])
response.message.content

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

from openai import OpenAI

client = OpenAI()

def chatbot(message, history):
    messages = [{"role": "system", "content": "you are a helpful assistant"}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=messages
        )
    
    return response.choices[0].message.content

# import gradio as gr
# gr.ChatInterface(fn=chatbot, type="messages").launch()

In [3]:
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI

In [5]:
import requests

url = "https://github.com/laxmimerit/All-CSV-ML-Data-Files-Download/raw/refs/heads/master/db_samples/Chinook.db"
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
response = requests.get(url)

if response.status_code == 200:
    with open("Chinook.db", "wb") as file:
        file.write(response.content)

    print("File downloaded successfully")

else:
    print("Failed to download the file")
    print(response.status_code)


print(db.get_usable_table_names())
db.run("SELECT * FROM album LIMIT 2")



File downloaded successfully
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2)]"

In [ ]:
from openai import OpenAI

# Initializes the ChatOpenAI model with a temperature of 0.7 (controlling randomness) and sets up the SQL toolkit.
model='gpt-4o-mini'

llm = ChatOpenAI(temperature=0.7, model_name=model)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
agent=create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [ ]:
## Use the SQL Agent
agent.invoke("How many employees are there?")

In [ ]:
agent.invoke("Which country's customers have made the most purchases?")

In [ ]:
agent.invoke("What are the top 5 countries by total purchases made?")

In [6]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [7]:
import glob 
import os
from openai import OpenAI

folders = glob.glob('myfiles')
folders

[]

In [9]:
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.pdf",
                             loader_cls=PyPDFLoader)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [19]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [13]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [20]:
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [21]:
db_name = "vector_db"
if os.path.exists(db_name):
    Chroma(persist_directory=db_name,
           embedding_function=embeddings).delete_collection()

In [22]:
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=embeddings,
                                    persist_directory=db_name)

ValueError: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.

In [23]:
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

NameError: name 'vectorstore' is not defined